# Задание 2. Итерационные методы решения линейных систем

In [1]:
from latexifier import latexify
from IPython.display import display, Markdown
import numpy as np
from typing import Callable, List, Tuple, Union, Any
import warnings
warnings.filterwarnings('ignore')

In [2]:
class Matrix(np.ndarray):
    """extends np.ndarray"""

    def __new__(cls, input_array):
        obj = np.asarray(input_array).view(cls)
        return obj

    def inverse_rows(self, i: int, j: int):
        """Swaps i-th and j-th rows of a matrix"""
        self[i], self[j] = self[j].copy(), self[i].copy()

    def inverse_cols(self, i: int, j: int):
        """Swaps i-th and j-th columns of a matrix"""
        self = self.transpose()
        self.inverse_rows(i, j)
        self = self.transpose()
    
    def tex(self) -> str:
        """Latexifies the matrix rounding up values up to 6 decimal values"""
        return latexify(self)

    def norm(self) -> float:
        """Calculates Matrix norm using np.linalg.norm"""
        return np.linalg.norm(self, 1)

    def inv(self) -> Any:
        return Matrix(np.linalg.inv(self))

def append(M: np.ndarray, N: np.ndarray) -> Matrix:
    """Appends two numpy arrays with axis=1"""
    return Matrix(np.append(arr=np.asarray(M), values=N, axis=1))

In [3]:
def Gauss(A: Matrix, B: Matrix, n: int) -> Matrix:
    """
    Вычисляет решения линейного уравнения Ax = B методом Гаусса с выбором
    главного элемента по столбцам (с перестановкой строк)
    """
    AB = append(A, B)
    X = [[0] for _ in range(n)]
    for k in range(n):
        j = k
        for i in range(k+1, n):
            if abs(AB[j][k]) < abs(AB[i][k]):
                j = i
        AB.inverse_rows(k, j)
        for j in range(k+1, n): 
            c = AB[j][k] / AB[k][k]
            for i in range(n+1):
                AB[j][i] -= c * AB[k][i]
    X[n-1][0] = AB[n-1][n] / AB[n-1][n-1]
    for k in range(n-1, -1, -1):
        s = 0
        for i in range(k+1, n):
            s += AB[k][i] * X[i][0]
        X[k][0] = (AB[k][n] - s) / AB[k][k]
    return Matrix(np.array(X))

In [4]:
def print_tex(*argv) -> None:
    """Displays a LaTex Markdown output"""
    res = ' '.join(['$$'] + [(latexify(arg) if isinstance(arg, np.ndarray) else str(arg)) for arg in argv] + ['$$'])
    display(Markdown(res))


def get_latex_column(arg: str, n: int) -> str:
    """
    Prepares a string with LaTex syntax for a column of `arg` with indeces form 1 to `n`
    """
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = arg + "_{" + str(i+1) + "}" + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def get_exact_column(col: Matrix, n: int) -> str:
    """Prepares a string with LaTex syntax for a column of {col} elements"""
    res = [r"""\begin{pmatrix}""", "\n"]
    for i in range(n):
        line = str(float(col[i][0])) + (r"\\" if i != n-1 else "") + "\n"
        res.append(line)
    res.append(r"""\end{pmatrix}""")
    return ''.join(res)


def print_system_output(A: Matrix, s: str, B: Matrix, X: Matrix, n: int, name: str) -> None:
    """Oupputs the result of solving a linear equation"""
    B_ = np.matmul(A, X)
    print_tex(r'\text{', name, '}~X^* = ', get_latex_column("x^*", n), '=', X.tex())
    print_tex(rf'{s} \times X^* = ', A.tex(), X.tex(), '=', B_.tex(), '= B^*')
    print_tex('B = ', get_exact_column(B, n), r'\stackrel{?}{=}', get_exact_column(B_, n), '= B^*')
    print_tex(r'B - B^* = ', get_exact_column(B - B_, n))

In [8]:
n = 4
# A = Matrix(np.random.rand(n, n))
# B = Matrix(np.random.rand(n, 1))
A = Matrix(np.array([
    [0.78, -0.02, -0.12, -0.14],
    [-0.02, 0.86, -0.04, 0.06],
    [-0.12, -0.04, 0.72, -0.08],
    [-0.14, 0.06, -0.08, 0.74]
]))
B = Matrix(np.array([
    [0.76],
    [0.08],
    [1.12],
    [0.68]
]))
print_tex('A=', A, ',~~', 'B=', B)

$$ A= \begin{pmatrix} 0.780000 & -0.020000 & -0.120000 & -0.140000 \\ -0.020000 & 0.860000 & -0.040000 & 0.060000 \\ -0.120000 & -0.040000 & 0.720000 & -0.080000 \\ -0.140000 & 0.060000 & -0.080000 & 0.740000 \end{pmatrix} ,~~ B= \begin{pmatrix} 0.760000 \\ 0.080000 \\ 1.120000 \\ 0.680000 \end{pmatrix} $$

In [9]:
X = Gauss(A, B, n)
print_system_output(A, 'A', B, X, n, 'Решение методом Гаусса:')

$$ \text{ Решение методом Гаусса: }~X^* =  \begin{pmatrix}
x^*_{1}\\
x^*_{2}\\
x^*_{3}\\
x^*_{4}
\end{pmatrix} = \begin{pmatrix} 1.534965 \\ 0.122010 \\ 1.975156 \\ 1.412955 \end{pmatrix} $$

$$ A \times X^* =  \begin{pmatrix} 0.780000 & -0.020000 & -0.120000 & -0.140000 \\ -0.020000 & 0.860000 & -0.040000 & 0.060000 \\ -0.120000 & -0.040000 & 0.720000 & -0.080000 \\ -0.140000 & 0.060000 & -0.080000 & 0.740000 \end{pmatrix} \begin{pmatrix} 1.534965 \\ 0.122010 \\ 1.975156 \\ 1.412955 \end{pmatrix} = \begin{pmatrix} 0.760000 \\ 0.080000 \\ 1.120000 \\ 0.680000 \end{pmatrix} = B^* $$

$$ B =  \begin{pmatrix}
0.76\\
0.08\\
1.12\\
0.68
\end{pmatrix} \stackrel{?}{=} \begin{pmatrix}
0.7599999999999999\\
0.07999999999999999\\
1.12\\
0.68
\end{pmatrix} = B^* $$

$$ B - B^* =  \begin{pmatrix}
1.1102230246251565e-16\\
1.3877787807814457e-17\\
0.0\\
0.0
\end{pmatrix} $$

In [10]:
D, D_inv, L, R = [Matrix(np.zeros((n, n))) for _ in range(4)]
for i in range(n):
    D[i][i] = A[i][i]
    D_inv[i][i] = 1 / A[i][i]
    for j in range(n):
        if j < i:
            L[i][j] = A[i][j]
        elif j > i:
            R[i][j] = A[i][j]
H = Matrix(np.identity(n) - D_inv @ A)
G = D_inv @ B
print_tex('H=', H, ',~~', 'G=', G)
print_tex(r'||H||_\infty = ', np.linalg.norm(H, np.inf))

$$ H= \begin{pmatrix} 0 & 0.025641 & 0.153846 & 0.179487 \\ 0.023256 & 0 & 0.046512 & -0.069767 \\ \frac{1}{6} & 0.055556 & 0 & \frac{1}{9} \\ 0.189189 & -0.081081 & 0.108108 & 0 \end{pmatrix} ,~~ G= \begin{pmatrix} 0.974359 \\ 0.093023 \\ \frac{14}{9} \\ 0.918919 \end{pmatrix} $$

$$ ||H||_\infty =  0.3783783783783784 $$

In [11]:
iter = 7

def iterative_method(H: Matrix = H, G: Matrix = G, i: int = i, n: int = n) -> Union[Matrix, Matrix]:
    out = Matrix(np.zeros((n, 1)))
    for _ in range(iter):
        # print_tex(H, G, out)
        prev = out
        out = H @ prev + G
    return out, prev

iter_ans, prev = iterative_method()
print_tex(r'\text{Решение методом простой итерации:}~', iter_ans)
print_tex(get_exact_column(X - iter_ans, n))

$$ \text{Решение методом простой итерации:}~ \begin{pmatrix} 1.534544 \\ 0.122011 \\ 1.974784 \\ 1.412563 \end{pmatrix} $$

$$ \begin{pmatrix}
0.00042125833343442665\\
-1.3266915103310462e-06\\
0.00037219061521898134\\
0.0003923703292800784
\end{pmatrix} $$